This code takes carbon emission and sequestration data across Y2Y and outputs a bar plot. Due to differences in data quality and processing methods throughout time, I only included years 2015-2023 for the annual comparison.

## Load packages and data

In [ ]:
# load packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# load y2y carbon sequestration data
y2y_df = pd.read_csv('./outputs/y2y_carbon_sequestration.csv')
y2y_df.info()

## Process data

In [ ]:
# filter df for annual emissions
y2y_emissions = y2y_df.filter(regex=r'gfw_forest_carbon_gross_emissions_\d{4}__Mg_C')
y2y_emissions.info()

In [ ]:
# Rename columns to just the year
y2y_emissions = y2y_emissions.rename(columns=lambda x: x.split("_")[5] if "gfw_forest_carbon_gross_emissions_" in x else x)
y2y_emissions.info()

In [ ]:
# Melt the DataFrame for plotting
y2y_emissions_melted = y2y_emissions.melt(var_name="Year", value_name="Emissions")

# Convert "Year" to an integer for proper ordering
y2y_emissions_melted["Year"] = y2y_emissions_melted["Year"].astype(int)

# divide emissions by 1e6 to get million tons
y2y_emissions_melted['Emissions'] = y2y_emissions_melted['Emissions'] / 1e6
y2y_emissions_melted

In [ ]:
# Compute a 3-year moving average
y2y_emissions_3yr = y2y_emissions_melted.groupby("Year", as_index=False)["Emissions"].mean()
y2y_emissions_3yr["Emissions"] = y2y_emissions_3yr["Emissions"].rolling(window=3, min_periods=1).mean()
y2y_emissions_3yr


In [ ]:
# create a df only 2015 onward
y2y_emissions_melted_2015 = y2y_emissions_melted.query('Year >= 2015')

In [ ]:
# save annual average sequestration in million tons
mean_seques = y2y_df['gfw_forest_carbon_average_annual_removals__Mg(C)_yr-1'].values.item() / 1e6
mean_seques

In [ ]:
# save annual average emissions in million tons
mean_emissions = y2y_df['gfw_forest_carbon_average_annual_emissions__Mg(C)_yr-1'].values.item() / 1e6
mean_emissions

## Create plots

I only used the plot from 2015 onward due to inconsistencies in data quality and processing methods pre and post 2015

In [ ]:
# Create a bar plot
plt.figure(figsize=(8, 5))
sns.barplot(x="Year", y="Emissions", data=y2y_emissions_melted, estimator=sum, errorbar=None, color="#BB5566")

# Add a dotted line at mean removals
plt.axhline(y=mean_seques, color='#004488', linestyle='dotted', linewidth=2)

# Add a dotted line at mean emissions
plt.axhline(y=mean_emissions, color='#000000', linestyle='dotted', linewidth=2)

# add a label for mean removals
plt.text(x=20, y=mean_seques+2, s="Mean Annual Sequestration", color='#004488', fontsize=10, ha='right')

# add a label for mean emissions
plt.text(x=14.2, y=mean_emissions+2, s="Mean Annual Emissions", color='#000000', fontsize=10, ha='right')

# Customize plot
plt.xlabel("Year")
plt.ylabel("Carbon Emissions (Million Tons)")
plt.title("Y2Y Annual Forest Carbon Emissions 2001-2023")
plt.xticks(rotation=45)
plt.yticks([0, 20, 40, 60, 80, 100])
plt.show()

In [ ]:
# Create a bar plot, only 2015 onward
plt.figure(figsize=(8, 5))
sns.barplot(x="Year", y="Emissions", data=y2y_emissions_melted_2015, estimator=sum, errorbar=None, color="#BB5566")

# Add a dotted line at mean removals
plt.axhline(y=mean_seques, color='#004488', linestyle='dotted', linewidth=2)

# Add a dotted line at mean emissions
#plt.axhline(y=mean_emissions, color='#000000', linestyle='dotted', linewidth=2)

# add a label for mean removals
plt.text(x=7.3, y=mean_seques+2, s="Mean Annual Removals", color='#004488', fontsize=10, ha='right')

# add a label for mean emissions
#plt.text(x=6.8, y=mean_emissions+2, s="Mean Annual Emissions", color='#000000', fontsize=10, ha='right')

# Customize plot
plt.xlabel("Year")
plt.ylabel("Carbon Emissions (Million Tons)")
plt.title("Y2Y Annual Forest Carbon Emissions 2015-2023")
plt.xticks(rotation=45)
plt.yticks([0, 20, 40, 60, 80, 100])
plt.show()